https://spell.ml/blog/pytorch-quantization-X8e7wBAAACIAHPhT

https://pytorch.org/tutorials/intermediate/quantized_transfer_learning_tutorial.html

https://zhuanlan-zhihu-com.translate.goog/p/299108528?_x_tr_sl=auto&_x_tr_tl=en&_x_tr_hl=en-US&_x_tr_pto=wapp

https://zhuanlan-zhihu-com.translate.goog/p/149659607?_x_tr_sl=auto&_x_tr_tl=en&_x_tr_hl=en-US&_x_tr_pto=wapp

https://github.com/IntelLabs/distiller/issues/327 -> Quantizing a MNIST

In [1]:
import numpy as np
import torch
import torch.nn as nn
import sys
from custom_convolve import convolve_torch
torch.set_printoptions(precision=30)
np.set_printoptions(precision=30)

### Test 1 Floating point model - Full Weight

In [4]:
class M_double(nn.Module):

    def __init__(self):
        super(M_double, self).__init__()
        # QuantStub converts tensors from floating point to quantized
        # self.BN = nn.BatchNorm2d(3)
        # self.conv = torch.nn.Conv2d(1, 1, 1)
        # self.conv = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)
        self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1).double()
        # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.03307433053851127625, -0.13484150171279907227, -0.21625524759292602539], 
        #                                                         [ 0.14247404038906097412, -0.14247404038906097412, -0.24932956695556640625], 
        #                                                         [ 0.32311078906059265137, -0.14501821994781494141, -0.21371106803417205811]]]]))
        # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 1.0, 1.0, -1.0], 
        #                                                         [ -1.0, 1.0, 1.0], 
        #                                                         [ 1.0, 1.0, -1.0]]]]).double())
        # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ -1.0, 0.0, 1.0], 
        #                                                         [ -1.0, 0.0, 1.0], 
        #                                                         [ -1.0, 0.0, 1.0]]]]))
        # self.conv.bias = torch.nn.Parameter(torch.tensor([1.0]))
        # self.relu = torch.nn.ReLU()
        # DeQuantStub converts tensors from quantized to floating point

    def forward(self, x):
        x = self.conv(x)
         
        return x

In [8]:
model_double = M_double()
model_double.eval()

M_double(
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [5]:
def main_loop(model_float, model_double):
    input_fp32 = torch.rand(1, 1, 300, 300)
    
    # model_output_float = model_float(input_fp32).detach()
    model_output_double = model_double(input_fp32.double()).detach()

    # print("Input image dtype", input_fp32.dtype)
    # convolved_img_numpy_float32 = convolve_numpy(input_fp32[0].detach().numpy().astype(np.float32), model_float.conv.weight.detach().numpy().astype(np.float32), model_float.conv.bias.detach().numpy().astype(np.float32))
    # # print('Numpy float 64')
    # convolved_img_numpy_double = convolve_numpy(input_fp32[0].detach().numpy().astype(np.float64), model_float.conv.weight.detach().numpy().astype(np.float64), model_float.conv.bias.detach().numpy().astype(np.float64))
    
    # difference_numpy_float32 = model_output_float.numpy() - convolved_img_numpy_float32
    # difference_numpy_double = model_output_double.numpy() - convolved_img_numpy_double

    # print("Numpy float model difference :", np.sum(difference_numpy_float32))
    # print("Numpy double model difference :", np.sum(difference_numpy_double))

    # convolved_img_torch_float = convolve_torch(input_fp32[0].detach(), model_float.conv.weight.detach(), model_float.conv.bias.detach())
    convolved_img_torch_double = convolve_torch(input_fp32[0].detach().double(), model_double.conv.weight.detach().double(), model_double.conv.bias.detach().double())
    
    # print("Outside convolved image shape", convolved_img.dtype)
    # print("Model Output type", model_output.dtype)
    
    # difference_torch_float32 = model_output_float - convolved_img_torch_float
    difference_torch_double = model_output_double - convolved_img_torch_double

    # print("Torch float model difference :", torch.sum(difference_torch_float32))
    print("Torch double model difference :", torch.sum(difference_torch_double))

In [9]:
for iter in range(10):
    input_fp32 = torch.rand(1, 1, 300, 300)
    
    # model_output_float = model_float(input_fp32).detach()
    model_output_double = model_double(input_fp32.double()).detach()

    # convolved_img_torch_float = convolve_torch(input_fp32[0].detach(), model_float.conv.weight.detach(), model_float.conv.bias.detach())
    convolved_img_torch_double = convolve_torch(input_fp32[0].detach().double(), model_double.conv.weight.detach().double(), model_double.conv.bias.detach().double())
    
    # print("Outside convolved image shape", convolved_img.dtype)
    # print("Model Output type", model_output.dtype)
    
    # difference_torch_float32 = model_output_float - convolved_img_torch_float
    difference_torch_double = model_output_double - convolved_img_torch_double

    # print("Torch float model difference :", torch.sum(difference_torch_float32))
    print("Torch double model difference :", torch.sum(difference_torch_double))
    
    print("Iteration :", iter)

Torch double model difference : tensor(1.082467449009527626913040876389e-14, dtype=torch.float64)
Iteration : 0
Torch double model difference : tensor(9.409140133698201680090278387070e-15, dtype=torch.float64)
Iteration : 1
Torch double model difference : tensor(2.609024107869117869995534420013e-15, dtype=torch.float64)
Iteration : 2
Torch double model difference : tensor(2.442490654175344388931989669800e-15, dtype=torch.float64)
Iteration : 3
Torch double model difference : tensor(2.942091015256664832122623920441e-15, dtype=torch.float64)
Iteration : 4
Torch double model difference : tensor(-6.911138328291599464137107133865e-15, dtype=torch.float64)
Iteration : 5
Torch double model difference : tensor(6.855627177060341637115925550461e-15, dtype=torch.float64)
Iteration : 6
Torch double model difference : tensor(4.107825191113079199567437171936e-15, dtype=torch.float64)
Iteration : 7
Torch double model difference : tensor(1.321165399303936283104121685028e-14, dtype=torch.float64)
Itera

##Stride Test

In [10]:
class M_stride(nn.Module):

    def __init__(self):
        super(M_stride, self).__init__()
        self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=2, padding=1)
        # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.03307433053851127625, -0.13484150171279907227, -0.21625524759292602539], 
        #                                                         [ 0.14247404038906097412, -0.14247404038906097412, -0.24932956695556640625], 
        #                                                         [ 0.32311078906059265137, -0.14501821994781494141, -0.21371106803417205811]]]]))
        # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 1.0, 1.0, -1.0], 
        #                                                         [ -1.0, 1.0, 1.0], 
        #                                                         [ 1.0, 1.0, -1.0]]]]).double())
        self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ -1.0, 0.0, 1.0], 
                                                              [ -1.0, 0.0, 1.0], 
                                                              [ -1.0, 0.0, 1.0]]]]).double())
        self.conv.bias = torch.nn.Parameter(torch.tensor([0.0]).double())
        # self.relu = torch.nn.ReLU()
        # DeQuantStub converts tensors from quantized to floating point

    def forward(self, x):
        x = self.conv(x)
         
        return x

In [11]:
model_fp32_stride = M_stride()
model_fp32_stride.eval()

M_stride(
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
)

In [12]:
model_fp32_stride(input_fp32.double())

tensor([[[[ 0.812963843345642089843750000000,  0.059485256671905517578125000000,
           -0.654202759265899658203125000000,  ...,
           -0.648304939270019531250000000000,  0.671665370464324951171875000000,
           -0.731793582439422607421875000000],
          [ 1.422105014324188232421875000000, -0.173482835292816162109375000000,
           -0.734119415283203125000000000000,  ...,
            0.136144816875457763671875000000, -0.165659725666046142578125000000,
           -0.928222298622131347656250000000],
          [ 1.116674184799194335937500000000,  0.160289287567138671875000000000,
           -0.155205786228179931640625000000,  ...,
            0.873344719409942626953125000000, -0.672932207584381103515625000000,
           -0.434129238128662109375000000000],
          ...,
          [ 1.382569432258605957031250000000,  0.005407094955444335937500000000,
            0.474898040294647216796875000000,  ...,
            0.139550626277923583984375000000, -0.57675814628601074218

In [13]:
result_stride = convolve_torch(input_fp32[0].detach().double(), model_fp32_stride.conv.weight.detach().double(), model_fp32_stride.conv.bias.detach().double(), stride=2)

In [14]:
result_stride

tensor([[[ 0.812963843345642089843750000000,  0.059485256671905517578125000000,
          -0.654202759265899658203125000000,  ...,
          -0.648304939270019531250000000000,  0.671665370464324951171875000000,
          -0.731793582439422607421875000000],
         [ 1.422105014324188232421875000000, -0.173482835292816162109375000000,
          -0.734119415283203125000000000000,  ...,
           0.136144816875457763671875000000, -0.165659725666046142578125000000,
          -0.928222298622131347656250000000],
         [ 1.116674184799194335937500000000,  0.160289287567138671875000000000,
          -0.155205786228179931640625000000,  ...,
           0.873344719409942626953125000000, -0.672932207584381103515625000000,
          -0.434129238128662109375000000000],
         ...,
         [ 1.382569432258605957031250000000,  0.005407094955444335937500000000,
           0.474898040294647216796875000000,  ...,
           0.139550626277923583984375000000, -0.576758146286010742187500000000,
    

In [15]:
model_fp32_stride(input_fp32.double())

tensor([[[[ 0.812963843345642089843750000000,  0.059485256671905517578125000000,
           -0.654202759265899658203125000000,  ...,
           -0.648304939270019531250000000000,  0.671665370464324951171875000000,
           -0.731793582439422607421875000000],
          [ 1.422105014324188232421875000000, -0.173482835292816162109375000000,
           -0.734119415283203125000000000000,  ...,
            0.136144816875457763671875000000, -0.165659725666046142578125000000,
           -0.928222298622131347656250000000],
          [ 1.116674184799194335937500000000,  0.160289287567138671875000000000,
           -0.155205786228179931640625000000,  ...,
            0.873344719409942626953125000000, -0.672932207584381103515625000000,
           -0.434129238128662109375000000000],
          ...,
          [ 1.382569432258605957031250000000,  0.005407094955444335937500000000,
            0.474898040294647216796875000000,  ...,
            0.139550626277923583984375000000, -0.57675814628601074218

In [16]:
torch.sum(result_stride - model_fp32_stride(input_fp32.double())[0].detach())

tensor(0., dtype=torch.float64)

## Test 3

In [26]:
class M_fullweight_multichannel(nn.Module):

    def __init__(self):
            super(M_fullweight_multichannel, self).__init__()
            # self.BN = nn.BatchNorm2d(3)
            # self.conv = torch.nn.Conv2d(1, 1, 1)
            # self.conv = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)
            self.conv = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1).double()
            # self.conv.weight = torch.nn.Parameter(torch.zeros(1,2, 3, 3))
            # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[0., 0., 0.],
            #                                                       [0., 1., 0.],
            #                                                       [0., 0., 0.]]],

            #                                                       [[[0., 0., 0.],
            #                                                         [0., 1., 0.],
            #                                                         [0., 0., 0.]]]]).double())                                          
            # self.conv.bias = torch.nn.Parameter(torch.tensor([0.0, 0.0]).double())
            # self.conv.bias = torch.nn.Parameter(torch.tensor([2.12300]))
            # self.relu = torch.nn.ReLU()
            

    def forward(self, x):
        # x = self.BN(x)
        x = self.conv(x)
        # x = self.relu(x)      
        
        return x

In [27]:
model_fullweight_multichannel = M_fullweight_multichannel()
model_fullweight_multichannel.eval()

M_fullweight_multichannel(
  (conv): Conv2d(3, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [28]:
model_fullweight_multichannel.conv.weight.detach()

tensor([[[[-0.044489271938800811767578125000, -0.013404434546828269958496093750,
            0.023243753239512443542480468750],
          [ 0.015496133826673030853271484375, -0.021780500188469886779785156250,
            0.130318447947502136230468750000],
          [-0.014340186491608619689941406250, -0.037016220390796661376953125000,
            0.007967264391481876373291015625]],

         [[-0.169396921992301940917968750000,  0.033638998866081237792968750000,
            0.085606716573238372802734375000],
          [-0.013384635560214519500732421875, -0.063823670148849487304687500000,
           -0.182716190814971923828125000000],
          [-0.161564111709594726562500000000,  0.057816185057163238525390625000,
            0.186273828148841857910156250000]],

         [[-0.025954263284802436828613281250, -0.144208341836929321289062500000,
            0.143235027790069580078125000000],
          [ 0.073251269757747650146484375000,  0.143819361925125122070312500000,
            0.06320

In [29]:
model_fullweight_multichannel.conv.weight.shape

torch.Size([2, 3, 3, 3])

In [30]:
input_fp32 = torch.rand(1, 3, 300, 300)

In [31]:
model_output_double = model_fullweight_multichannel(input_fp32.double()).detach()

In [32]:
convolved_img_torch_double = convolve_torch(input_fp32[0].detach().double(), model_fullweight_multichannel.conv.weight.detach().double(), model_fullweight_multichannel.conv.bias.detach().double())

In [33]:
print("Torch double model difference :", torch.sum(model_output_double - convolved_img_torch_double))

Torch double model difference : tensor(-2.386979502944086561910808086395e-15, dtype=torch.float64)


### Test 4: With image 

In [44]:
import skimage
# Reading the image
#img = skimage.io.imread("fruits2.png")
img = skimage.data.chelsea()
img = img/255;
# Converting the image into gray.
# img = skimage.color.rgb2gray(img)

In [45]:
img_tensor = torch.tensor(img).double()

In [46]:
img_tensor.shape

torch.Size([300, 451, 3])

In [47]:
img_tensor.resize_(1, img.shape[2], img.shape[0], img.shape[1])

tensor([[[[0.560784313725490202173773468530, 0.470588235294117640528099855146,
           0.407843137254901955124353207793,  ...,
           0.431372549019607864906333816180, 0.305882352941176494098840521474,
           0.619607843137254921117573758238],
          [0.439215686274509797826226531470, 0.337254901960784336800713845150,
           0.599999999999999977795539507497,  ...,
           0.298039215686274505667796574926, 0.623529411764705887577520115883,
           0.470588235294117640528099855146],
          [0.317647058823529393478679594409, 0.647058823529411797359500724269,
           0.494117647058823550310080463532,  ...,
           0.176470588235294129075825253494, 0.105882352941176469118822467408,
           0.050980392156862744390544150974],
          ...,
          [0.756862745098039191304906125879, 0.678431372549019640061374047946,
           0.682352941176470606521320405591,  ...,
           0.454901960784313719177163193308, 0.219607843137254898913113265735,
           

In [48]:
img_tensor.shape

torch.Size([1, 3, 300, 451])

In [49]:
model_fullweight_multichannel(input_fp32.double()).detach()

tensor([[[[ 0.009264791328643717793767109470,  0.020349141884572818916865344363,
            0.247532056323970017785995878512,  ...,
            0.362966044249569352864170923567,  0.312273167569265985843429689339,
           -0.086619392947608009336590839666],
          [ 0.314880183421353598838265952509,  0.281343919261858310676416294882,
           -0.384865522588665076586522673097,  ...,
            0.076644306999684719272636357346, -0.129412370510200047668547540525,
           -0.053456211045836588002799771857],
          [ 0.127945480619844564884601822996,  0.013465282656756338841574915932,
            0.308128685710179650580897714462,  ...,
            0.130484165896261106176723387762,  0.120841194782762062498449040504,
           -0.214429652549148752616048341224],
          ...,
          [ 0.416846653738015715529030558173,  0.054413661788379252826075571647,
           -0.187454820082911310930029458177,  ...,
            0.027758782060248454737205747733, -0.17145649762667347637

In [50]:
convolved_img_torch_double = convolve_torch(input_fp32[0].detach().double(), model_fullweight_multichannel.conv.weight.detach().double(), model_fullweight_multichannel.conv.bias.detach().double())

In [51]:
print("Torch double model difference :", torch.sum(model_output_double - convolved_img_torch_double))

Torch double model difference : tensor(-2.386979502944086561910808086395e-15, dtype=torch.float64)
